# PSF Matching Kernels

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import importlib

from mophongo.psf_map import PSFRegionMap
import mophongo.psf
importlib.reload(mophongo.psf)
from mophongo.psf import DrizzlePSF, PSF
from mophongo.kernels import KernelLookup
from astropy.io import fits


## Build the PSF region map

In [ ]:
field = 'uds-test'
filt = 'F444W'
data_dir = Path('data')
drz_file = data_dir / f'{field}-{filt.lower()}_sci.fits'
csv_file = data_dir / f'{field}-{filt.lower()}_wcs.csv'

dpsf = DrizzlePSF(driz_image=str(drz_file), csv_file=str(csv_file))
footprints = {k: v for i, (k, v) in enumerate(dpsf.footprint.items()) if i < 10}
prm = PSFRegionMap.from_footprints(footprints, buffer_tol=1.0/3600, area_factor=300)

fig, ax = plt.subplots()
prm.regions.plot(column='psf_key', ax=ax, edgecolor='k', cmap='tab20')
ax.set_xlabel('RA'); ax.set_ylabel('Dec')


## Create PSF cubes for both filters

In [ ]:
filters = ['F444W', 'F770W']
psf_cubes = {}
for filt in filters:
    drz_file = data_dir / f'{field}-{filt.lower()}_sci.fits'
    csv_file = data_dir / f'{field}-{filt.lower()}_wcs.csv'
    dpsf = DrizzlePSF(driz_image=str(drz_file), csv_file=str(csv_file))
    centroids = prm.regions.geometry.centroid
    cube = []
    for ra, dec in zip(centroids.x, centroids.y):
        cube.append(dpsf.get_psf(ra, dec, filt))
    cube = np.asarray(cube)
    fits.writeto(data_dir / f'{field}_{filt.lower()}_psf.fits', cube, overwrite=True)
    psf_cubes[filt] = cube


## Compute matching kernels

In [ ]:
cube_444 = psf_cubes['F444W']
cube_770 = psf_cubes['F770W']
kernels = []
for p444, p770 in zip(cube_444, cube_770):
    ker = PSF.from_array(p444).matching_kernel(PSF.from_array(p770))
    kernels.append(ker)
kernels = np.asarray(kernels)
fits.writeto(data_dir / f'{field}_kernel_f444w_to_f770w.fits', kernels, overwrite=True)


## Fast kernel recall

In [ ]:
i = 1
ra, dec = prm.regions.loc[i].geometry.centroid.xy
psf_key = prm.regions.loc[i].psf_key
print(psf_key,ra[0],dec[0])
prm.lookup_key(ra[0], dec[0])

# psf_key is an index into the kernels array and matches the order of the regions
kernel_at_radec = kernels[psf_key]


